In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.svm
from sklearn.tree import DecisionTreeClassifier 
import sklearn.ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer

In [2]:
import pandas as pd

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self 
    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = preprocessing.LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = preprocessing.LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [3]:
data = np.loadtxt('ThoraricSurgery.arff',dtype = 'str',delimiter=',',comments='@')
columns = np.arange(17)
df = pd.DataFrame(data,columns=columns)
df = MultiColumnLabelEncoder(columns = [0,3,4,5,6,7,8,9,10,11,12,13,14,16]).fit_transform(df)
data = df.values
data = data.astype(np.float)
X_train, X_test, y_train, y_test = train_test_split(
    data[:,:16],data[:,16], test_size=0.33, random_state=42)

In [4]:
#Nearest Neighbours
knearest_neighbours_classifier = KNeighborsClassifier(n_neighbors=2, algorithm='ball_tree')
knearest_neighbours_classifier.fit(X_train, y_train);
y_pred = knearest_neighbours_classifier.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8333333333333334

In [5]:
#SVM
svm_model = sklearn.svm.SVC(kernel = 'linear')
svm_model.fit(X_train, y_train);
y_pred = svm_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8333333333333334

In [6]:
#Decision Tree Classifier
decision_tree_model = DecisionTreeClassifier(random_state=0)
decision_tree_model.fit(X_train, y_train);
y_pred = decision_tree_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.7628205128205128

In [7]:
#Random Forest Classifier
random_forest_classifier = sklearn.ensemble.RandomForestClassifier(random_state=0,n_estimators=9)
random_forest_classifier.fit(X_train, y_train);
y_pred = random_forest_classifier.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8141025641025641

In [8]:
#Ada Boost Classifier
adaboost_classifier = sklearn.ensemble.AdaBoostClassifier(n_estimators=9, random_state=0,algorithm='SAMME')
adaboost_classifier.fit(X_train, y_train);
y_pred = adaboost_classifier.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8269230769230769

In [13]:
#Logistic Regression Classifier
logistic_model = sklearn.linear_model.LogisticRegression(fit_intercept=True, penalty='l1', solver='liblinear',max_iter = 1000)
logistic_model.fit(X_train, y_train);
y_pred = logistic_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.8333333333333334

In [14]:
#Gaussian naive Bayes classification
naive_bayes_classifier = GaussianNB()
naive_bayes_classifier.fit(X_train, y_train);
y_pred = naive_bayes_classifier.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.1794871794871795